In [1]:
import numpy as np
import pandas as pd

In [23]:
from siphon.catalog import TDSCatalog
from datetime import datetime, timedelta
from wwo_hist import retrieve_hist_data
import xarray as xr
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [12]:
def get_weather_forecast(start, end):
  """прогноз погоды глобальной модели GFS
  """
  tz = 'Europe/Moscow'
  forecastvariables = ['Temperature_isobaric', 'Temperature_surface', 'Precipitation_rate_surface']

  gfs_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog'
                       '/grib/NCEP/GFS/Global_0p5deg/catalog.xml')
  latest = gfs_cat.latest
  ncss = latest.subset()
  query = ncss.query().variables(*forecastvariables)
  query.time_range(start, end).accept('netCDF4')
  nc = ncss.get_data(query)
  return nc


weather_frcst = get_weather_forecast(datetime.utcnow() + timedelta(days=1), datetime.utcnow() + timedelta(days=11))

In [13]:
weather_frcst

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Originating_or_generating_Center: US National Weather Service, National Centres for Environmental Prediction (NCEP)
    Originating_or_generating_Subcenter: 0
    GRIB_table_version: 2,1
    Type_of_generating_process: Forecast
    Analysis_or_forecast_generating_process_identifier_defined_by_originating_centre: Analysis from GFS (Global Forecast System)
    Conventions: CF-1.6
    history: Read using CDM IOSP GribCollection v3
    featureType: GRID
    History: Translated to CF-1.0 Conventions by Netcdf-Java CDM (CFGridWriter2)
Original Dataset = /data/ldm/pub/native/grid/NCEP/GFS/Global_0p5deg/GFS_Global_0p5deg_20201122_0000.grib2.ncx3#LatLon_361X720-p25S-180p0E; Translation Date = 2020-11-22T10:56:44.881Z
    geospatial_lat_min: -90.0
    geospatial_lat_max: 90.0
    geospatial_lon_min: 0.0
    geospatial_lon_max: 359.5
    dimensions(sizes): time2(80), lat(361), lon(720), isobaric6(34)
    var

In [24]:
targets = ('06005', '06022', '06296', '06027', '05004', '05012', '05024', '05805')
import xarray as xr
asunp = gpd.read_file('http://asunp.meteo.ru/geoits-rest/services/asunp/geo.json', driver='GeoJSON')
posts = asunp[asunp.gidro.isin(targets) & (asunp.ktoCategory.isin(['post_gidro', 'station_gidro']))].sort_values(by=['lon']).set_index('gidro', drop=False)
fds = xr.open_dataset(xr.backends.NetCDF4DataStore(weather_frcst))
fds = fds.sel(isobaric6=100000)

def predict(weather_frcst):
  predictions = []
  for identifier in targets:
    point = weather_frcst.sel(lat=posts.loc[identifier].lat,
                              lon=posts.loc[identifier].lon,
                              method='nearest'
                              )
    future = point.to_dataframe().reset_index()
    future = future.rename({i: "ds" for i in future.columns if i.startswith("time")}, axis=1)
    future = future.rename({
        'Temperature_surface': 'temperature_ground',
        'Temperature_isobaric': 'temperature_air',
        'Precipitation_rate_surface': 'precipitation_amount'
        }, axis=1)

    future = future.set_index('ds').resample('D')\
      .agg({
        'precipitation_amount': 'sum',
        'temperature_air': 'mean', 'temperature_ground': 'mean'
    })
    
    future = future.reset_index()
    future['temperature_air'] = future.temperature_air - 273.15
    future['temperature_ground'] = future.temperature_ground - 273.15

    future['spring'] = future['ds'].apply(is_spring)
    future['summer'] = future['ds'].apply(is_summer)

    with open(f'./models/{int(identifier)}.pkl', 'rb') as f:
      forecaster = pickle.load(f)
      forecast = forecaster.predict(future)
      sf = forecast.set_index('ds').yhat.rename('5012')
      predictions.append(sf)

  return predictions

NameError: name 'gpd' is not defined

In [21]:
predictions = predict(fds)

NameError: name 'posts' is not defined